In [84]:
import requests
import json

# Standard Data Science Helpers
import numpy as np
import pandas as pd
import scipy

import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
#init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import os
from IPython.display import Image, display, HTML

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import cv2
import matplotlib.pyplot as plt

%run login.py

loginquery = f"""
mutation {{
  logIn(
      email:\"{login}\",
      password:\"{pwd}\") {{
    jwt {{
      token
      exp
    }}
  }}
}}
"""

url = 'https://api.numina.co/graphql'
mylogin = requests.post(url, json={'query': loginquery})
token = mylogin.json()['data']['logIn']['jwt']['token']

devices = {'Streetscape - Sandbox': 'SWLSANDBOX1', 'Outside - Sandbox': 'SWLSANDBOX3',
           'Under Raincoat - Sandbox': 'SWLSANDBOX2'}

In [81]:
def getHeatmapImage(imgpath, token=token):
    # Heatmaps
    query = """
    query {
      feedHeatmaps(
        serialno: "SWLSANDBOX1",
        startTime:"2019-11-03T00:00:00",
        endTime:"2019-12-31T00:00:00",
        objClasses:["pedestrian"],
        timezone:"America/New_York") {
        edges {
          node {
            time
            objClass
            heatmap
          }
        }
      }
    }
    """

    heatdata = requests.post(url, json={'query': query}, headers = {'Authorization':token})
    
    heatarr = np.zeros((480,640))
    
    for point in heatdata.json()['data']['feedHeatmaps']['edges'][0]['node']['heatmap']:
        arr[point[1]][point[0]] = point[2]
        
    img = cv2.imread(imgpath)
    heatmap = cv2.resize(arr, (img.shape[1], img.shape[0]))
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    combined_image = cv2.addWeighted(img,1,heatmap,0.3,0)
    
    return combined_image
    